**[2] Get busy times & bad weather**





*   Google Places API key
*   Weather API Key



In [40]:
import requests

In [41]:
GOOGLE_API_KEY = "Your_GOOGLE_PLACES_API_key"
WEATHER_API_KEY = 'Your_OPENWEATHER_API_key'
place_id = 'ChIJPYSDLXWBwokRHLcHIl02Kh8'



*    Get Village busy times from GMaps API



In [42]:
place_details_url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=name,opening_hours&key={GOOGLE_API_KEY}"

def get_detailed_opening_hours():
    try:
        response = requests.get(place_details_url)
        response.raise_for_status()
        data = response.json()

        if data.get('result', {}).get('opening_hours'):
            opening_hours = data['result']['opening_hours'].get('weekday_text', 'Not available')
            print("Opening Hours:")
            if isinstance(opening_hours, list):
                for day in opening_hours:
                    print(f"   {day}")
            else:
                print("No detailed hours available.")
        else:
            print("No detailed opening hours found for this place.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

get_detailed_opening_hours()

Opening Hours:
   Monday: 11:30 AM – 3:00 PM, 5:00 – 10:00 PM
   Tuesday: Closed
   Wednesday: 11:30 AM – 3:00 PM, 5:00 – 10:00 PM
   Thursday: 11:30 AM – 3:00 PM, 5:00 – 10:00 PM
   Friday: 11:30 AM – 3:00 PM, 5:00 – 10:00 PM
   Saturday: 11:30 AM – 3:00 PM, 5:00 – 9:50 PM
   Sunday: 11:30 AM – 3:00 PM, 5:00 – 9:50 PM


In [43]:
place_details_url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=name,opening_hours&key={GOOGLE_API_KEY}"

def time_to_minutes(time_str):
    """Convert time in HHMM format to total minutes"""
    hours = int(time_str[:2])
    minutes = int(time_str[2:])
    return hours * 60 + minutes

def get_average_busy_time():
    try:
        response = requests.get(place_details_url)
        response.raise_for_status()
        data = response.json()

        if data.get('result', {}).get('opening_hours'):
            opening_hours = data['result']['opening_hours'].get('periods', [])
            if opening_hours:
                total_open_hours = 0
                total_days = 0
                day_open_hours = {}

                for period in opening_hours:
                    day = period['open']['day']
                    open_time = period['open']['time']
                    close_time = period['close']['time']

                    open_minutes = time_to_minutes(open_time)
                    close_minutes = time_to_minutes(close_time)

                    open_duration = close_minutes - open_minutes
                    if open_duration < 0:
                        open_duration += 24 * 60

                    if day not in day_open_hours:
                        day_open_hours[day] = 0
                    day_open_hours[day] += open_duration

                for day, total_minutes in day_open_hours.items():
                    hours = total_minutes // 60
                    minutes = total_minutes % 60

                    total_open_hours += total_minutes
                    total_days += 1

                if total_days > 0:
                    average_open_hours = (total_open_hours / total_days) / 60  # Convert minutes to hours
                    print(f"\nAverage open hours per day: {average_open_hours:.2f} hours")
                else:
                    print("No opening hours data found for this place.")
            else:
                print("No periods available in the opening hours.")
        else:
            print("No detailed opening hours found for this place.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

get_average_busy_time()



Average open hours per day: 8.44 hours




*   Get temperature & rain near Village



In [46]:
place_details_url_weather = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=name,geometry&key={GOOGLE_API_KEY}"

def get_coordinates():
    """Get the latitude and longitude of the place."""
    try:
        response = requests.get(place_details_url_weather)
        response.raise_for_status()
        data = response.json()

        if data.get('result', {}).get('geometry', {}).get('location'):
            location = data['result']['geometry']['location']
            return location['lat'], location['lng']
        else:
            print("Coordinates not found.")
            return None, None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching place details: {e}")
        return None, None

def get_weather(lat, lng):
    """Get temperature and rain data using OpenWeather API."""
    weather_url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lng}&units=metric&appid={WEATHER_API_KEY}"
    try:
        response = requests.get(weather_url)
        response.raise_for_status()
        data = response.json()

        temperature = data['main']['temp']
        rain = data.get('rain', {}).get('1h', 0)  # Get rain volume in the last hour, if available
        return temperature, rain
    except requests.exceptions.RequestException as e:
        print(f"Error fetching weather data: {e}")
        return None, None

def get_temperature_and_rain():
    lat, lng = get_coordinates()

    if lat and lng:
        temperature, rain = get_weather(lat, lng)
        farenheit = temperature * 9/5 + 32
        if temperature is not None:
            print(f"Current Temperature: {temperature}°C or {farenheit}°F ")
        else:
            print("Could not fetch temperature data.")

        if rain is not None:
            print(f"Rain Volume in the last hour: {rain} mm")
        else:
            print("No rain data available in the last hour.")
    else:
        print("Unable to fetch location data.")

get_temperature_and_rain()


Current Temperature: 4.65°C or 40.370000000000005°F 
Rain Volume in the last hour: 2.37 mm
